In [1]:
import wfdb
import numpy as np
import os
import scipy.io
import tensorflow as tf
from tensorflow import  keras
from keras.callbacks import TensorBoard
from sklearn.preprocessing import MinMaxScaler,StandardScaler,scale
from skimage.measure import block_reduce
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,LSTM, MaxPooling1D, Dense, Flatten, Dropout, GlobalMaxPooling1D,BatchNormalization
from keras.optimizers import Adam

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
def function(muscle,disease, label_map):
    k=0
    for i in range (len(label_map)):
        if muscle in label_map[i]:
            if disease in label_map[i]:
                k=k+1
    return k

In [3]:
def scale_data_standard(X_train,X_test,X_val):
    #divide by variance and subtract mean for data 
    scaler = StandardScaler()
    data = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    X_val = scaler.transform(X_val)
    return X_train ,X_test ,X_val

In [4]:
def gen_tensors_list(data,dim):
    dataset  = list()
    for i in range (len(data)):
        arg = tf.convert_to_tensor(data[i], dtype=tf.float32)
        arg = tf.reshape(arg,dim)
        dataset.append(arg)

    return dataset

In [5]:
def windowing_fun(data,window_size=23437,overlap=14062):
    windowed_data = [data[j][i : i + window_size] for j in range(0,len(data)) for i in range(0, len(data[j]), window_size-overlap)]
    final_data = [windowed_data[i] for i in range(len(windowed_data)) if len(windowed_data[i]) == window_size ]
    return final_data

In [6]:
def remove_nan_rows(arr,Y):
    # Find the rows with NaN values
    nan_rows = np.isnan(arr).any(axis=1)

    # Convert the boolean array to integer indices
    index_to_remove = np.where(nan_rows)[0]

    # Remove the rows with NaN values
    arr = np.delete(arr, index_to_remove, axis=0)
    Y = np.delete(Y, index_to_remove, axis=0)

    return arr,Y

In [7]:
def noise_for_samples(data, Y):
    data = np.array(data)
    noise = np.random.normal(0, 1, size=data.shape)
    noisy_data = np.add(data, noise)
    data = np.concatenate((data, noisy_data))
    Y = np.concatenate((Y, Y))
    return noisy_data 

In [8]:
def preprocessing(X_train,X_val,X_test,Y_train,Y_val,Y_test):
    
    X_train = windowing_fun(X_train, 23437)
    X_val = windowing_fun(X_val, 23437)
    X_test = windowing_fun(X_test, 23437)

   
    X_train,Y_train = remove_nan_rows(X_train,Y_train)
    X_val,Y_val = remove_nan_rows(X_val,Y_val)
    X_test,Y_test = remove_nan_rows(X_test,Y_test)
    
    X_train ,X_test ,X_val = scale_data_standard(X_train,X_test,X_val)
    
    tensors_list_train = gen_tensors_list(X_train,(11719,1))
    tensors_list_val = gen_tensors_list(X_val,(11719,1))
    tensors_list_test = gen_tensors_list(X_test,(11719,1))
    
    return tensors_list_train,tensors_list_val,tensors_list_test,Y_train,Y_val,Y_test

In [9]:
def outputs(Y, X):
    
    last_index_als = len(Y) - 1 - Y[::-1].index(0)
    last_index_normal = len(Y) - 1 - Y[::-1].index(1)
    last_index_myopathie = len(Y) - 1 - Y[::-1].index(2)
    
    windowed_als = len(windowing_fun(X[:last_index_als+1],23437))
    windowed_normal = len(windowing_fun(X[last_index_als+1:last_index_normal+1],23437))
    windowed_myopathie = len(windowing_fun(X[last_index_normal+1:],23437))
    return windowed_normal, windowed_als, windowed_myopathie

In [10]:
def last_index (Y):
    last_index_normal = len(Y) - 1 - Y[::-1].index(1)
    last_index_als = len(Y) - 1 - Y[::-1].index(0)
    last_index_myopathie = len(Y) - 1 - Y[::-1].index(2)
    return last_index_normal, last_index_als, last_index_myopathie

In [11]:
def generate_dataframe(data,files):
    df = pd.DataFrame([], columns=['type_signal',"num_personne","muscle","num_enregistrement","signal"])
    windowed_data = [{"signal": data[j],"type_signal":files[j][5],"num_personne": files[j][6:8],"muscle": files[j][8:10],"num_enregistrement":files[j][10:12] } for j in range(0,len(data))]
    final_data = [windowed_data[i] for i in range(len(windowed_data))]
    df =df.append(final_data, ignore_index=True)
    #df =pd.concat([df, final_data])
    #df = pd.concat([df, pd.DataFrame.from_records(final_data)], ignore_index=True)
    return df

In [12]:
def NaN_values(signal_data_array):
    # Check if there are any NaN values in the data
    nan_indices = np.argwhere(np.isnan(signal_data_array))

    # Check if there are any empty values in the data
    empty_indices = np.argwhere(np.array([len(x) == 0 for x in signal_data_array]))
    np.set_printoptions(threshold=np.inf)

     #Print the indices of any NaN or empty values
    if len(nan_indices) > 0:
        print("Found NaN values ")
    else:
        print("none")
    if len(empty_indices) > 0:
        print("Found empty values ")
    else:
        print("none")
    nan_indices.shape

In [13]:
import os
import wfdb
import numpy as np

database_dir = '/kaggle/input/emglab'

hea_files = [f for f in os.listdir(database_dir)]

# Initialize empty arrays to store the signal data and record names
signal_data_list = []
record_name_list = []
# Loop over each .hea file and read its signal data
for file in hea_files:
    if file.endswith(".hea"):
        record_name = os.path.splitext(file)[0] # Get the record name from the file name
        record_path = os.path.join(database_dir, record_name) # Get the full path to the record
        record = wfdb.rdrecord(record_path) # Read the record
        signal_data = np.array(record.p_signal) # Get the signal data as a numpy array
        signal_data_list.append(signal_data) # Append the signal data to the list
        record_name_list.append(record_name) # Append the record name to the list

# Convert the list of signal data arrays to a 3D numpy array
signal_data_array = np.stack(signal_data_list)

print(record_name_list)


['N2001C03BB51', 'N2001C07BB51', 'N2001M02BB51', 'N2001C06BB65', 'N2001C07BB63', 'N2001C07BB76', 'N2001A03BB66', 'N2001C09BB51', 'N2001A04BB62', 'N2001C01BB61', 'N2001M03BB03', 'N2001M05BB60', 'N2001M07BB20', 'N2001M02BB61', 'N2001C06BB51', 'N2001A04BB60', 'N2001M03BB10', 'N2001A06BB11', 'N2001C03BB80', 'N2001C04BB73', 'N2001C09BB67', 'N2001C06BB57', 'N2001C04BB66', 'N2001M03BB08', 'N2001M07BB10', 'N2001A04BB66', 'N2001M02BB06', 'N2001A06BB14', 'N2001C06BB61', 'N2001C03BB64', 'N2001A06BB09', 'N2001A07BB02', 'N2001M03BB04', 'N2001C07BB57', 'N2001C09BB59', 'N2001A04BB51', 'N2001C04BB52', 'N2001C02BB76', 'N2001A06BB07', 'N2001C06BB73', 'N2001C01BB74', 'N2001M02BB59', 'N2001C04BB63', 'N2001C09BB57', 'N2001C02BB70', 'N2001C05BB54', 'N2001M04BB52', 'N2001C02BB65', 'N2001C06BB77', 'N2001A06BB18', 'N2001C07BB66', 'N2001C01BB68', 'N2001C08BB60', 'N2001C07BB59', 'N2001M04BB54', 'N2001A03BB54', 'N2001A01BB05', 'N2001A03BB51', 'N2001C08BB80', 'N2001A08BB03', 'N2001C01BB59', 'N2001C09BB78', 'N2001C

In [14]:
f=function('BB','N2001M' , record_name_list) + function('BB','N2001A' , record_name_list)
print('number of signals taken from sick people from the biceps brachii muscle ',f)
f=function('BB','N2001C' , record_name_list)
print('number of signals taken from normal people from the biceps brachii muscle ',f)

number of signals taken from sick people from the biceps brachii muscle  205
number of signals taken from normal people from the biceps brachii muscle  270


In [15]:
signal_data_array.shape

(475, 262124, 1)

In [16]:
# Combine the labels and signals arrays into a list of tuples
data = list(zip(record_name_list, signal_data_array))

# Sort the list of tuples by the first element (the labels)
sorted_data = sorted(data, key=lambda x: x[0])

# Extract the sorted signals into a new array
signal_data_array = [x[1] for x in sorted_data]

record_name_list = [x[0] for x in sorted_data]

In [17]:
#redo balancing
function('BB','N2001C' , record_name_list[98:368])
function('BB','N2001C' , record_name_list[303:368])

65

In [18]:
signal_data_array = np.delete(signal_data_array,np.s_[290:368], axis=0)

In [19]:
record_name_list = np.delete(record_name_list,np.s_[290:368], axis=0)

In [20]:
f=function('BB','N2001A' , record_name_list)
print('number of signals taken from patients with ALS from the biceps brachii muscle ',f)
f=function('BB','N2001M' , record_name_list)
print('number of signals taken from patients with myopathy from the biceps brachii muscle ',f)
f=function('BB','N2001C' , record_name_list)
print('number of signals taken from normal people from the biceps brachii muscle ',f)

number of signals taken from patients with ALS from the biceps brachii muscle  98
number of signals taken from patients with myopathy from the biceps brachii muscle  107
number of signals taken from normal people from the biceps brachii muscle  192


In [21]:
print(np.shape(signal_data_array))
print(np.shape(record_name_list))

(397, 262124, 1)
(397,)


In [22]:
#check if there are NaN values
NaN_values(signal_data_array)

Found NaN values 
none


In [23]:
df = generate_dataframe(signal_data_array,record_name_list)

/tmp/ipykernel_29/879706199.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df =df.append(final_data, ignore_index=True)


In [24]:
df

,type_signal,num_personne,muscle,num_enregistrement,signal
0,A,01,BB,01,"[[144.19775692378118], [145.3421835660334], [1..."
1,A,01,BB,02,"[[-5.41695277332723], [0.30518043793392846], [..."
2,A,01,BB,03,"[[-19.15007248035401], [-19.607843137254903], ..."
3,A,01,BB,04,"[[2.2125581750209813], [3.9673456931410698], [..."
4,A,01,BB,05,"[[20.752269779507134], [19.378957808804458], [..."
...,...,...,...,...,...
392,M,07,BB,18,"[[1.9073777370870528], [2.059967956054017], [3..."
393,M,07,BB,19,"[[-8.010986495765621], [-4.425116350041963], [..."
394,M,07,BB,20,"[[13.885709925993744], [10.986495765621424], [..."
395,M,07,BB,21,"[[7.324330510414283], [7.782101167315176], [8...."


In [25]:
def f(row):
    return {"C":1,"M":2, "A":0}[row["type_signal"]]

df["num_class"] =df.apply(f, axis=1) 

In [26]:
df

,type_signal,num_personne,muscle,num_enregistrement,signal,num_class
0,A,01,BB,01,"[[144.19775692378118], [145.3421835660334], [1...",0
1,A,01,BB,02,"[[-5.41695277332723], [0.30518043793392846], [...",0
2,A,01,BB,03,"[[-19.15007248035401], [-19.607843137254903], ...",0
3,A,01,BB,04,"[[2.2125581750209813], [3.9673456931410698], [...",0
4,A,01,BB,05,"[[20.752269779507134], [19.378957808804458], [...",0
...,...,...,...,...,...,...
392,M,07,BB,18,"[[1.9073777370870528], [2.059967956054017], [3...",2
393,M,07,BB,19,"[[-8.010986495765621], [-4.425116350041963], [...",2
394,M,07,BB,20,"[[13.885709925993744], [10.986495765621424], [...",2
395,M,07,BB,21,"[[7.324330510414283], [7.782101167315176], [8....",2


In [27]:
del signal_data_array,record_name_list

In [28]:
for signal in df['type_signal'].unique():
    print("",f"signal {signal}:", sep="\n")
    print("pat count")
    print(df[df['type_signal']==signal]['num_personne'].value_counts())


signal A:
pat count
03    20
04    20
06    18
08    15
01    12
07    10
05     3
Name: num_personne, dtype: int64

signal C:
pat count
01    30
02    30
03    30
04    30
05    30
06    30
07    12
Name: num_personne, dtype: int64

signal M:
pat count
02    25
07    22
03    16
05    16
06    16
04    12
Name: num_personne, dtype: int64


In [29]:
df_train = df[~df["num_personne"].isin(["06","05"])]

print(df_train["num_personne"].value_counts())
print(df_train['type_signal'].value_counts())

03    66
04    62
02    55
07    44
01    42
08    15
Name: num_personne, dtype: int64
C    132
A     77
M     75
Name: type_signal, dtype: int64


In [30]:
df_val = df[df["num_personne"] == "06"]

df_val['type_signal'].value_counts()

C    30
A    18
M    16
Name: type_signal, dtype: int64

In [31]:
df_test = df[df["num_personne"] == "05"]

df_test['type_signal'].value_counts()

C    30
M    16
A     3
Name: type_signal, dtype: int64

In [32]:
X_train = df_train['signal'].tolist()
Y_train = df_train['num_class'].tolist()
print(Y_train)
print(np.asarray(Y_train).shape)
print(np.asarray(Y_train).shape)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
(284,)
(284,)


In [33]:
X_val = df_val['signal'].tolist()
Y_val = df_val['num_class'].tolist()
print(Y_val)
print(np.asarray(Y_val).shape)
print(np.asarray(X_val).shape)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
(64,)
(64, 262124, 1)


In [34]:
X_test = df_test['signal'].tolist()
Y_test = df_test['num_class'].tolist()
print(Y_test)
print(np.asarray(Y_test).shape)
print(np.asarray(X_test).shape)

[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
(49,)
(49, 262124, 1)


In [35]:
last_index_normal, last_index_als, last_index_myopathie = last_index (Y_train)
last_index_normal, last_index_als, last_index_myopathie

(208, 76, 283)

In [36]:
del df,df_train,df_test

In [37]:
windowed_normal_train,windowed_als_train,windowed_myopathie_train = outputs(Y_train, X_train)
windowed_normal_val,windowed_als_val,windowed_myopathie_val = outputs(Y_val, X_val)
windowed_normal_test,windowed_als_test,windowed_myopathie_test = outputs(Y_test, X_test)

In [38]:
Y_train = [0]*(windowed_als_train)+[1]*(windowed_normal_train)+[2]*(windowed_myopathie_train)
print(np.asarray(Y_train).shape)
Y_val = [0]*(windowed_als_val)+[1]*(windowed_normal_val)+[2]*(windowed_myopathie_val)
print(np.asarray(Y_val).shape)
Y_test = [0]*(windowed_als_test)+[1]*(windowed_normal_test)+[2]*(windowed_myopathie_test)
print(np.asarray(Y_test).shape)

(7384,)
(1664,)
(1274,)


In [39]:
print(np.asarray(X_train).shape)
print(np.asarray(X_val).shape)
print(np.asarray(X_test).shape)

(284, 262124, 1)
(64, 262124, 1)
(49, 262124, 1)


In [40]:
X_train = windowing_fun(X_train, 23437)
X_val = windowing_fun(X_val, 23437)
X_test = windowing_fun(X_test, 23437)
print(np.shape(X_test))
print(np.shape(X_train))

print(np.shape(X_val))
print(np.shape(X_val))

print(np.shape(Y_test))
print(np.shape(Y_train))

(1274, 23437, 1)
(7384, 23437, 1)
(1664, 23437, 1)
(1664, 23437, 1)
(1274,)
(7384,)


In [41]:
X_train,Y_train = remove_nan_rows(X_train,Y_train)
X_val,Y_val = remove_nan_rows(X_val,Y_val)
X_test,Y_test = remove_nan_rows(X_test,Y_test)

print(np.shape(X_test))
print(np.shape(X_train))

print(np.shape(X_val))
print(np.shape(Y_val))

print(np.shape(Y_test))
print(np.shape(Y_train))

(1269, 23437, 1)
(7349, 23437, 1)
(1655, 23437, 1)
(1655,)
(1269,)
(7349,)


In [42]:
'''X_train = noise_for_samples(X_train, Y_train)
X_test  = noise_for_samples(X_test, Y_test)
X_val  = noise_for_samples(X_val, Y_val)

X_train = noise_for_samples(X_train, Y_train)
X_test  = noise_for_samples(X_test, Y_test)
X_val  = noise_for_samples(X_val, Y_val)'''

'X_train = noise_for_samples(X_train, Y_train)\nX_test  = noise_for_samples(X_test, Y_test)\nX_val  = noise_for_samples(X_val, Y_val)\n\nX_train = noise_for_samples(X_train, Y_train)\nX_test  = noise_for_samples(X_test, Y_test)\nX_val  = noise_for_samples(X_val, Y_val)'

In [43]:
Y_train = list(Y_train)
last_index_normal, last_index_als, last_index_myopathie = last_index (Y_train)
last_index_normal, last_index_als, last_index_myopathie

(5405, 1973, 7348)

In [44]:
# Convert Y_train_resampled to a pandas Series object
Y_train_series = pd.Series(Y_train)

# Check the new class distribution
print(Y_train_series.value_counts())

1    3432
0    1974
2    1943
dtype: int64


In [45]:
X_train_als = X_train[:last_index_als + 1]
X_train_normal = X_train[last_index_als + 1:last_index_normal + 1]
X_train_myopathy = X_train[last_index_normal + 1:]

In [46]:
num_samples_als, num_timesteps_als, num_channels_als = X_train_als.shape
X_train_als = X_train_als.reshape(num_samples_als, num_channels_als * num_timesteps_als)


num_samples_normal, num_timesteps_normal, num_channels_normal = X_train_normal.shape
X_train_normal = X_train_normal.reshape(num_samples_normal, num_channels_normal * num_timesteps_normal)


num_samples_myopathy, num_timesteps_myopathy, num_channels_myopathy = X_train_myopathy.shape
X_train_myopathy = X_train_myopathy.reshape(num_samples_myopathy, num_channels_myopathy * num_timesteps_myopathy)
X_train_normal.shape,X_train_als.shape,X_train_myopathy.shape

((3432, 23437), (1974, 23437), (1943, 23437))

In [47]:
from imblearn.over_sampling import SMOTE
normal_als = np.concatenate((X_train_normal,X_train_als))
normal_myopathy = np.concatenate((X_train_normal,X_train_myopathy))
Y_normal_als = [1] * len(X_train_normal) + [0] * len(X_train_als)
Y_normal_myopathy = [1] * len(X_train_normal) + [2] * len(X_train_myopathy)

# Apply SMOTE to the training set
smote = SMOTE()
normal_als, Y_normal_als = smote.fit_resample(normal_als, Y_normal_als)

# Apply SMOTE to the training set
smote = SMOTE()
normal_myopathy, Y_normal_myopathy = smote.fit_resample(normal_myopathy, Y_normal_myopathy)





In [48]:
# Convert Y_train_resampled to a pandas Series object
Y_train_series = pd.Series(Y_normal_als)

# Check the new class distribution
print(Y_train_series.value_counts())

# Convert Y_train_resampled to a pandas Series object
Y_train_series = pd.Series(Y_normal_myopathy)

# Check the new class distribution
print(Y_train_series.value_counts())

1    3432
0    3432
dtype: int64
1    3432
2    3432
dtype: int64


In [49]:
def first_last_index(arr):
    first_index = -1
    last_index = -1
    for i in range(len(arr)):
        if arr[i] == 1:
            if first_index == -1:
                first_index = i
            last_index = i
    return first_index, last_index

In [50]:
first_last_index(Y_normal_als)

(0, 3431)

In [51]:
Y_train_als = Y_normal_als[3430:]
X_train_als = normal_als[3430:]



In [52]:
X_train = np.concatenate((X_train_als,normal_myopathy))
Y_train = np.concatenate((Y_train_als,Y_normal_myopathy))

In [53]:
print(Y_train)

[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [54]:
# Convert Y_train_resampled to a pandas Series object
Y_train_series = pd.Series(Y_train)

# Check the new class distribution
print(Y_train_series.value_counts())

1    3434
0    3432
2    3432
dtype: int64


In [55]:
X_train = gen_tensors_list(X_train,(23437,1))
X_val = gen_tensors_list(X_val,(23437,1))
X_test = gen_tensors_list(X_test,(23437,1))

In [56]:
print(np.shape(X_test))
print(np.shape(X_train))
print(np.shape(X_val))

print(np.shape(Y_test))
print(np.shape(Y_train))
print(np.shape(Y_val))

(1269, 23437, 1)
(10298, 23437, 1)
(1655, 23437, 1)
(1269,)
(10298,)
(1655,)


In [57]:
from keras.regularizers import l2
model = Sequential([
    
    #Conv1D(#of filters, filters size, activation function)
    #in keras for the first layer we always need to mention the input shape
    Conv1D(32,  kernel_size=13, padding='valid', dilation_rate=1,activation = 'relu', input_shape = (23437,1),kernel_initializer='he_normal'), 
    
    BatchNormalization(),
    Dropout(0.2),
    #LAYER2
    Conv1D(64, kernel_size=11, padding='valid' , dilation_rate=2, activation = 'relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.0001)),
    
    BatchNormalization(),
 
    Dropout(0.3),
    #LAYER3
    Conv1D(128, kernel_size=9, padding='valid', dilation_rate=3, activation = 'relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.001)),
    
    BatchNormalization(),
    
    Dropout(0.4),
    
    Conv1D(256, kernel_size=7, padding='valid', dilation_rate=4, activation = 'relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.01)),
    
    BatchNormalization(),
    
    #GlobalMaxPooling1D layer
    GlobalMaxPooling1D(data_format = 'channels_last'),
    
   # Flatten(),
    
    #FC Layers:
    
    #LAYER4
    #Dense : fully connected  Dense(#neurons, activation function )
    
    Dense(128, activation = 'relu'),
    
    BatchNormalization(),
    
    Dropout(0.3),
    
    #LAYER5
    Dense(3, activation = 'softmax')
    
    
])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 23425, 32)         448       
                                                                 
 batch_normalization (BatchN  (None, 23425, 32)        128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 23425, 32)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 23405, 64)         22592     
                                                                 
 batch_normalization_1 (Batc  (None, 23405, 64)        256       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 23405, 64)         0

In [ ]:
# Set the initial learning rate
initial_lr = 0.001

# Create the Adam optimizer
opt = Adam(learning_rate = initial_lr, beta_1=0.9, beta_2=0.999, amsgrad=False, epsilon=None)

# Create the ReduceLROnPlateau callback to decrease the learning rate by a factor of 10
# when the validation loss doesn't improve for 5 consecutive epochs
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=6, verbose=1, min_lr=1e-10)

# Create the EarlyStopping callback to stop training if the validation loss doesn't improve
# for 20 consecutive epochs
early_stop = EarlyStopping(monitor='val_loss', patience=25, verbose=1)

# Compile the model
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

# Train the model with early stopping
history = model.fit(
    
    x = np.asarray(X_train),
    y = np.asarray(Y_train),
    #validation_split=0.2,
    validation_data = (np.asarray(X_val), np.asarray(Y_val)),
    shuffle=True,
    epochs=100,
    batch_size=8,
    callbacks=[reduce_lr, early_stop]

)

Epoch 1/100


In [ ]:

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

# Plot training & validation accuracy values
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()
  
print(history.history.keys())


In [ ]:
score = model.evaluate(np.asarray(X_test),np.asarray (Y_test) ) 

In [ ]:
model.save("ALSnet patient 3 for test (norm).h5")

In [ ]:
y_pred = model.predict(np.asarray(X_test))

In [ ]:
y_predicted_labels = [{0:"Neuropathie",1:"Normal",2:"muopathy"}[i] for i in np.argmax(y_pred,axis=1)]

In [ ]:
y_true_labels = [{0:"Neuropathie",1:"Normal",2:"myopathy"}[i] for i in Y_test]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

con_mat = tf.math.confusion_matrix(labels=Y_test, predictions=np.argmax(y_pred,axis=1)).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = ["Neuropathie","Normal","Myopathie"], 
                     columns = ["Neuropathie","Normal","Myopathie"])

figure = plt.figure(figsize=(5,5))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

predictions=np.argmax(y_pred,axis=1)
# y_true is the ground truth labels
# y_pred is the predicted labels
print(classification_report(Y_test, predictions))
